 <h1 style="text-align: center; color: RED">AB Testi ile Bidding Yöntemlerinin
Dönüşümünün Karşılaştırılması</h1>

<span style="color: blue; font-weight: bold;">1-İŞ PROBLEMİ</span>

Facebook kısa süre önce mevcut "maximumbidding" adı verilen
teklif verme türüne alternatif olarak yeni bir teklif türü olan
"average bidding"’i tanıttı.
Müşterilerimizden biri olan bombabomba.com, bu yeni özelliği test
etmeye karar verdi ve average bidding'inmaximumbidding'den
daha fazla dönüşüm getirip getirmediğini anlamak için bir A/B
testi yapmak istiyor.
A/B testi 1 aydır devam ediyor ve bombabomba.com şimdi sizden
bu A/B testinin sonuçlarını analiz etmenizi bekliyor.
Bombabomba.com için nihai başarı ölçütü Purchase'dır. Bu
nedenle, istatistiksel testler için Purchase metriğine
odaklanılmalıdır.


<span style="color: blue; font-weight: bold;">2-VERİ SETİ HİKAYESİ</span>

Bir firmanın web site bilgilerini içeren bu veri setinde kullanıcıların gördükleri ve tıkladıkları reklam sayıları gibi bilgilerin yanı sıra
buradan gelen kazanç bilgileri yer almaktadır. Kontrol ve Test grubu olmak üzere iki ayrı veri seti vardır. Bu veri setleri
ab_testing.xlsx excel’inin ayrı sayfalarında yer almaktadır. Kontrol grubuna Maximum Bidding, test grubuna Average
Bidding uygulanmıştır.

**Impression**: Reklam görüntüleme sayısı

**Click**: Görüntülenen reklama tıklama sayısı

**Purchase**: Tıklanan reklamlar sonrası satın alınan ürün sayısı

**Earning**: Satın alınan ürünler sonrası elde edilen kazanç

 <h1 style="text-align: center; color: RED">PROJE GÖREVLERİ</h1>

In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.stats.api as sms
from scipy.stats import shapiro, levene, ttest_ind

import itertools
import seaborn as sns
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, pearsonr, spearmanr, kendalltau, \
    f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
#pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

<span style="color: blue; font-weight: bold;">Görev 1: Veriyi Hazırlama ve Analiz Etme</span>

**Adım1 :** ab_testing_data.xlsx adlı kontrol ve test grubu verilerinden oluşan veri setini okutunuz. Kontrol ve test grubu verilerini ayrı
değişkenlere atayınız.

In [28]:
control_group = pd.read_excel("C:\\Users\\ASLI\\Desktop\\Data Scientist Bootcamp\\4.Measurement Problems\\datasets\\ab_testing.xlsx",sheet_name = 'Control Group' )

In [30]:
test_group = pd.read_excel("C:\\Users\\ASLI\\Desktop\\Data Scientist Bootcamp\\4.Measurement Problems\\datasets\\ab_testing.xlsx",sheet_name = 'Test Group' )

**Adım 2:** Kontrol ve test grubu verilerini analiz ediniz

In [32]:
control_group.head()

,Impression,Click,Purchase,Earning
0,82529.459271,6090.077317,665.211255,2311.277143
1,98050.451926,3382.861786,315.084895,1742.806855
2,82696.023549,4167.965750,458.083738,1797.827447
3,109914.400398,4910.882240,487.090773,1696.229178
4,108457.762630,5987.655811,441.034050,1543.720179


In [35]:
control_group.shape

(40, 4)

In [37]:
test_group.head()

,Impression,Click,Purchase,Earning
0,120103.503796,3216.547958,702.160346,1939.611243
1,134775.943363,3635.082422,834.054286,2929.405820
2,107806.620788,3057.143560,422.934258,2526.244877
3,116445.275526,4650.473911,429.033535,2281.428574
4,145082.516838,5201.387724,749.860442,2781.697521


In [39]:
test_group.shape

(40, 4)

In [41]:
test_group.describe().T#test grubunun ortalama izlenme  ve maddi getirisi daha yüksek görünüyor.

,count,mean,std,min,25%,50%,75%,max
Impression,40.0,120512.411758,18807.448712,79033.834921,112691.970770,119291.300775,132050.578933,158605.920483
Click,40.0,3967.549761,923.095073,1836.629861,3376.819024,3931.359804,4660.497911,6019.695079
Purchase,40.0,582.106097,161.152513,311.629515,444.626828,551.355732,699.862360,889.910460
Earning,40.0,2514.890733,282.730852,1939.611243,2280.537426,2544.666107,2761.545405,3171.489708


In [43]:
control_group.describe().T

,count,mean,std,min,25%,50%,75%,max
Impression,40.0,101711.449068,20302.157862,45475.942965,85726.690349,99790.701078,115212.816543,147539.336329
Click,40.0,5100.657373,1329.985498,2189.753157,4124.304129,5001.220602,5923.803596,7959.125069
Purchase,40.0,550.894059,134.108201,267.028943,470.095533,531.206307,637.957088,801.795020
Earning,40.0,1908.568300,302.917783,1253.989525,1685.847205,1975.160522,2119.802784,2497.295218


**Adım 3:** Analiz işleminden sonra concat metodunu kullanarak kontrol ve test grubu verilerini birleştiriniz.

In [54]:
control_group['Group'] = 'Control'
test_group['Group'] = 'Test'

In [56]:
df = pd.concat([control_group,test_group], axis=0, ignore_index=True)

In [58]:
df.head()

,Impression,Click,Purchase,Earning,Group
0,82529.459271,6090.077317,665.211255,2311.277143,Control
1,98050.451926,3382.861786,315.084895,1742.806855,Control
2,82696.023549,4167.965750,458.083738,1797.827447,Control
3,109914.400398,4910.882240,487.090773,1696.229178,Control
4,108457.762630,5987.655811,441.034050,1543.720179,Control


In [60]:
df.tail()

,Impression,Click,Purchase,Earning,Group
75,79234.911929,6002.213585,382.047116,2277.863984,Test
76,130702.239410,3626.320072,449.824592,2530.841327,Test
77,116481.873365,4702.782468,472.453725,2597.917632,Test
78,79033.834921,4495.428177,425.359102,2595.857880,Test
79,102257.454089,4800.068321,521.310729,2967.518390,Test


<span style="color: blue; font-weight: bold;">Görev 2: A/B Testinin Hipotezinin Tanımlanması</span>

**Adım 1**: Hipotezi tanımlayınız.

H0: m1=m2 (control ve test gruplarının ortalama kazançları arasında istatistiksel olarak fark yoktur.)

H1: m1!=m2 (fark yoktur)

**Adım 2:** Kontrol ve test grubu için purchase (kazanç) ortalamalarını analiz ediniz.

In [75]:
df.groupby('Group').agg({'Purchase':'mean'})# ortalamalar arasında fark var görünüyor. Aba istatistiksel olarak bu fark gercekten var mı?

,Purchase
Group,
Control,550.89406
Test,582.10610


<span style="color: blue; font-weight: bold;">Görev 3: Hipotez Testinin Gerçekleştirilmesi</span>

**Adım 1:** Hipotez testi yapılmadan önce varsayım kontrollerini yapınız.
    
Bunlar Normallik Varsayımı ve Varyans Homojenliğidir. Kontrol ve test grubunun normallik varsayımına uyup uymadığını Purchase değişkeni
üzerinden ayrı ayrı test ediniz.

Normallik Varsayımı :

H0: Normal dağılım varsayımı sağlanmaktadır.

H1: Normal dağılım varsayımı sağlanmamaktadır.

p < 0.05 H0 RED , p > 0.05 H0 REDDEDİLEMEZ

Test sonucuna göre normallik varsayımı kontrol ve test grupları için sağlanıyor mu ? Elde edilen p-value değerlerini yorumlayınız.

Varyans Homojenliği :

H0: Varyanslar homojendir.

H1: Varyanslar homojen Değildir.

p < 0.05 H0 RED , p > 0.05 H0 REDDEDİLEMEZ

Kontrol ve test grubu için varyans homojenliğinin sağlanıp sağlanmadığını Purchase değişkeni üzerinden test ediniz.

Test sonucuna göre normallik varsayımı sağlanıyor mu? Elde edilen p-value değerlerini yorumlayınız.


In [88]:
#Normallik Varsayımı
test_stat, pvalue= shapiro(df.loc[df['Group']=='Control', 'Purchase'])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9773, p-value = 0.5891


* p value değeri 0.05'ten büyük oldugu için H0 reddedilemez. Yani normallik varsayımı sağlanmaktadır.

In [91]:
#Normallik Varsayımı
test_stat, pvalue= shapiro(df.loc[df['Group']=='Test', 'Purchase'])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9589, p-value = 0.1541


* p value değeri 0.05'ten büyük oldugu için H0 reddedilemez. Yani normallik varsayımı sağlanmaktadır.

In [103]:
#Varyans Homojenliği
test_stat, pvalue= levene(df.loc[df['Group']=='Test', 'Purchase'],
                          df.loc[df['Group']=='Control', 'Purchase'])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 2.6393, p-value = 0.1083


* p value değeri 0.05'ten büyük oldugu için H0 reddedilemez. Yanivaryans homojenliği sağlanmaktadır.

**Adım 2:** Normallik Varsayımı ve Varyans Homojenliği sonuçlarına göre uygun testi seçiniz.

* Her iki  varsayımda sağlandığı için parametrik olan bağımsız iki örneklem t testi yapılacaktır.

**Adım 3:** Test sonucunda elde edilen p_value değerini göz önünde bulundurarak kontrol ve test grubu satın alma ortalamaları arasında istatistiki
olarak anlamlı bir fark olup olmadığını yorumlayınız.

In [114]:
#T testi
test_stat, pvalue = ttest_ind(df.loc[df['Group']=='Test', 'Purchase'],
                              df.loc[df['Group']=='Control', 'Purchase'],
                              equal_var=True)
print('Test Stat = %.4f, p-value=%.4f' % (test_stat,pvalue))

Test Stat = 0.9416, p-value=0.3493


* p value değeri 0.05 den büyük oldugu için Ho reddeilemez yani purchase değişkeninde iki grup için anlamlı bir fark yoktur.

<span style="color: blue; font-weight: bold;">Görev 4: Sonuçların Analizi</span>

**Adım 1:** Hangi testi kullandınız, sebeplerini belirtiniz.

Normallik varsayımı yaparken shapiro testini, varyans homojenliği testi yaparken levene tesi kullandık.

Her iki varsayımda sağlandığı için iki bağımsız örneklem t testini kullandık.

**Adım 2:** Elde ettiğiniz test sonuçlarına göre müşteriye tavsiyede bulununuz.

Sonuçlara bakıldıgında AVERAGE BIDDING'IN getirdiği satın alma ortlamaları ile
eski ürün MAX BIDDING'ın getirdiği ortalama arasında bir fark yoktur. Bu test yapılırken 40 gözlem içeren bir veri srti ile yapıldı. Veri srtindeki gözlem sayısı artırılarak sonuclar tekrardan analiz edilebilir. Yeni ürün ile eski ürünü test etmek için biraz daha zaman harcanabilir.Eğer test için süre yoksa mevcut uygulama ile devam edilebilir. 